In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import Series, DataFrame

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
'''
Create a dataset exemple for decision tree.

returns:
    dataset: 
    labels:
'''

def createDataset():
#     dataset = DataFrame([[1, 1, 'yes'],
#                         [1, 1, 'yes'],
#                         [1, 0, 'no'],
#                         [0, 1, 'no'],
#                         [0, 1, 'no']], 
#                         columns = ['no surfacing', 'flippers', 'fish'])
    
    dataset = DataFrame([[0, 0, 0, 0, 'no'], 
            [0, 0, 0, 1, 'no'],
            [0, 1, 0, 1, 'yes'],
            [0, 1, 1, 0, 'yes'],
            [0, 0, 0, 0, 'no'],
            [1, 0, 0, 0, 'no'],
            [1, 0, 0, 1, 'no'],
            [1, 1, 1, 1, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [2, 0, 1, 2, 'yes'],
            [2, 0, 1, 1, 'yes'],
            [2, 1, 0, 1, 'yes'],
            [2, 1, 0, 2, 'yes'],
            [2, 0, 0, 0, 'no']],
            columns = ['年龄', '有工作', '有自己的房子', '信贷情况', '类别'])

    return dataset

In [3]:
'''
Calculate the Shannon entropy.

Calculate the Shannon entropy of the input dataset, in which 
the last column must be the label of each instance. Then return 
the shannon_entropy.

parameters: 
    dataset: a DataFrame, the last column must be the label 
        of instance.
    
Returns: 
    shannonEnt: float. 
'''

def calcShannonEnt(dataset):
    label_counts = dataset.iloc[:, -1].value_counts()  # 计算类标签不同值出现的次数
    shannon_entropy = 0.0
    for label in label_counts.index:
        prob = float(label_counts[label]) / len(dataset)
        shannon_entropy -= prob * np.log2(prob)
    return shannon_entropy

# 利用pandas的广播特性直接对整个Series做除法，然后将prob和做点积并加上负号，即求得香农熵
# def calcShannonEnt(dataset):
#     label_counts = dataset.iloc[:, -1].value_counts()  # 计算类标签不同值出现的次数
#     shannon_entropy = 0.0
    
#     prob = label_counts.astype(float) / len(my_dataset)
#     shannon_entropy -= prob.dot(np.log2(prob))
#     return shannon_entropy

In [4]:
my_dataset = createDataset()
calcShannonEnt(my_dataset)

0.9709505944546686

In [5]:
'''
Spliting the dataset.

Deleting the best feature column of input dataset, and return rows, 
in which the best feature column's Series.values equal with input 
parameter value. 

Parameters:
    dataset: DataFrame.
    feature: label of DataFrame.
    value: the value of the Series 'dataset[feature]'.

Returns:
    retDataset: DataFrame.
    
Exemple:
    my_dataset
    no surfacing	flippers	label
    1	1	yes
    1	1	yes
    1	0	no
    0	1	no
    0	1	no
    
    my_dataset[my_dataset['flippers'] == 1]
    no surfacing	flippers	label
    1	1	yes
    1	1	yes
    0	1	no
    0	1	no
    
    my_dataset[my_dataset['flippers'] == 0]
    no surfacing	flippers	label
    1	0	no
'''

def splitDataset(dataset, feature, value):
    # dataset[dataset[feature] == value]利用布尔索引获取最优特征下不同取值所在的行。
    retDataset = dataset[dataset[feature] == value].drop(labels=feature, axis='columns')
    return retDataset

In [6]:
'''
Choose the best feature to split.

Parameters:
    dataset: DataFrame.
        The last column should be the label of each instance.
        
Returns:
    best_feature:
        The best feature DataFrame.columns.
'''

def chooseBestFeatureToSplit(dataset):
    entropy = calcShannonEnt(dataset)
    best_info_gain = 0.0
    best_feature = 0
    for feature in dataset.columns[:-1]:  # 计算每个特征的信息增益，并将最优特征存入best_feature
        unique_values = dataset[feature].unique()  # 特征的每一个可能取值
        conditional_entropy = 0.0
        for value in unique_values:  # 计算当前特征的条件熵
            sub_dataset = splitDataset(dataset, feature, value)  # 当前特征将dataset划分为unique_values个子集
            prob = len(sub_dataset) / len(dataset)
            '''
            calcShannonEnt在计算香农熵时已经带了负号，
            因此此处计算条件熵时不能用-=。
            conditional_entropy -= prob * calcShannonEnt(sub_dataset)
            '''
            conditional_entropy += prob * calcShannonEnt(sub_dataset)
        info_gain = entropy - conditional_entropy  #当前特征的信息增益
        if info_gain > best_info_gain:  #更新最优特征
            best_info_gain = info_gain
            best_feature = feature
    return best_feature
        


In [7]:
'''
Return the majority class of current dataset.

This function is used when all features are iterated and 
the columns of dataset only have class_label. Then decision 
tree returns the majority count class as leaf node.

Parameters:
    dataset: DataFrame.
        Only have the class_label column.
        
Returns:
    majority_class:
        Majority count value of class_label.

'''

def majorityCount(dataset):
    # 对class label计数并降序排列，返回第一个index。
    class_count = dataset.iloc[:, -1].value_counts().sort_values(ascending=False)
    majority_class = class_count.index[0]
    return majority_class

In [8]:
'''
Create decision tree.

Parameters:
    dataset: DataFrame.
        
Returns:
    decision_tree: a python dict.

'''

def createTree(dataset):
    if dataset.iloc[:, -1].nunique() == 1:  # 只剩一个类别，停止划分
        return dataset.iloc[:, -1].unique()[0]  # 返回类标签的具体值，即叶子节点
    if len(dataset.columns) == 1:  # 遍历完所有特征，此时dataset中只剩下label_class列
        return majorityCount(dataset)
    best_feature = chooseBestFeatureToSplit(dataset)
    '''
    使用python字典构建决策树，
    e.g. decision_tree = {best_feature: {}}
    decision_tree[best_feature]为上式中的{}，
    decision_tree[best_feature][value] = createTree(sub_dataset)在{}中生成子节点，
    其中value为最优特征的取值，它将数据集分为dataset[best_feature].unique()个子集。
    在这些子集中递归调用createTree。
    如'flippers': {1: 'yes', 0: 'no'}中，
    最优特征有两个取值1和0，因此'flippers'之下有两个子集，
    在此例中，两个子集中均为一个类别，所以这两个子集分别生成一个叶子节点'yes','no'。
    '''
    decision_tree = {best_feature: {}}
    # 在最优特征下递归调用createTree，继续生成内部节点，直到满足两个if终止条件，生成叶子节点
    for value in dataset[best_feature].unique():  
        sub_dataset = splitDataset(dataset, best_feature, value)
        decision_tree[best_feature][value] = createTree(sub_dataset)
    return decision_tree


In [9]:
createTree(my_dataset)

{'有自己的房子': {0: {'有工作': {0: 'no', 1: 'yes'}}, 1: 'yes'}}